In [2]:
from pymongo import MongoClient

# Replace these with your actual MongoDB connection details
client = MongoClient('mongodb://172.16.101.171:27017/')
db = client['LLM']  # Replace with your database name
collection = db['CureDoc_Evaluation']  # Replace with your collection name

# Define the aggregation pipeline
pipeline = [
    {
        '$match': {
            'Name': 'SpecificDiseaseName'  # Replace with the actual disease name you're querying
        }
    },
    {
        '$unwind': '$entries'
    },
    {
        '$unwind': '$entries.Doc1.Cd_rating'  # Adjust for each doctor you need to average
    },
    {
        '$group': {
            '_id': '$entries.Doc1',  # Adjust the grouping criteria based on your data structure
            'avgClarity': {'$avg': '$entries.Doc1.Cd_rating.Clarity'},
            'avgAccuracy': {'$avg': '$entries.Doc1.Cd_rating.Accuracy'},
            'avgBrevity': {'$avg': '$entries.Doc1.Cd_rating.Brevity'},
            'avgRelevance': {'$avg': '$entries.Doc1.Cd_rating.Relevance'},
            # Add other fields as necessary
        }
    }
]

# Execute the aggregation pipeline
results = collection.aggregate(pipeline)
print(results)
# Print the results
for result in results:
    print(result)
    print('here')


In [11]:
from pymongo import MongoClient

# Replace these with your actual MongoDB connection details
client = MongoClient('mongodb://172.16.101.171:27017/')
db = client['LLM']  # Replace with your database name
collection = db['CureDoc_Evaluation']  

# Specific disease name you're interested in
disease_name = "Diabetes treatment questions-Dr Ali"  # Example disease name, replace with your target

# Aggregation pipeline adjusted for nested structure
pipeline = [
    {
        '$match': {
            'Name': disease_name  # Filters documents by disease name
        }
    },
    {
        '$unwind': '$entries'  # Deconstructs the entries array
    },
    {
        '$project': {
            'entries': 1,
            '_id': 0  # Exclude the _id field from the results
        }
    }
]

# Execute the aggregation pipeline
results = collection.aggregate(pipeline)

# Helper function to calculate average rating
def calculate_average_rating(ratings):
    rating_keys = ['Clarity', 'Accuracy', 'Brevity', 'Relevance']
    averages = {}
    for key in rating_keys:
        # Filter out non-numeric values and calculate average
        numeric_values = [rating[key] for rating in ratings if isinstance(rating.get(key), (int, float)) and rating.get(key) is not None]
        if numeric_values:
            averages[key] = sum(numeric_values) / len(numeric_values)
        else:
            averages[key] = None  # Handle case where no ratings are available
    return averages

# Processing results
doc_ratings = {'Doc1': [], 'Doc2': [], 'Doc3': []}  # Adjust based on the number of doctors
gpt_ratings = {'Doc1': [], 'Doc2': [], 'Doc3': []} 
for entry in results:
    for doc in doc_ratings.keys():
        if doc in entry['entries']:
            doc_ratings[doc].append(entry['entries'][doc]['gpt_rating'])

# Calculate average ratings for each doctor
for doc, ratings in doc_ratings.items():
    average_ratings = calculate_average_rating(ratings)
    print(f"{doc} average ratings: {average_ratings}")

print('Model 2')

# for entry in results:
#     for doc in gpt_ratings.keys():
#         if doc in entry['entries']:
#             gpt_ratings[doc].append(entry['entries'][doc]['gpt_rating'])

# # Calculate average ratings for each doctor
# for doc, ratings in gpt_ratings.items():
#     average_ratings = calculate_average_rating(ratings)
#     print(f"{doc} average ratings: {average_ratings}")

Doc1 average ratings: {'Clarity': 0.0, 'Accuracy': 0.0, 'Brevity': 0.0, 'Relevance': 0.0}
Doc2 average ratings: {'Clarity': 0.375, 'Accuracy': 0.875, 'Brevity': 0.875, 'Relevance': 0.125}
Doc3 average ratings: {'Clarity': 8.875, 'Accuracy': 8.75, 'Brevity': 7.25, 'Relevance': 9.3125}
Model 2
